In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
from datetime import datetime

In [2]:
sp = pd.read_csv('final_model_data_V2.csv')

In [3]:
sp

,gvkey,month,year,date,ggroup,gsubind,conm,dpr,PEG_trailing,bm,...,US Population,Bank Prime Loan Rate,M2,"Bank Credit, All Commercial Banks",Currency in Circulation,10-Year Treasury Constant Maturity Minus Federal Funds Rate,Rent Inflation Rate,US GDP Growth Rate,trt1m,moving_risk
0,9783.0,12.0,1997.0,1998-01-31,4010.0,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,0.000849,0.0,-0.008832,-0.001071,-0.001059,0.000000,0.249390,0.011473,-8.2076,NaN
1,9783.0,1.0,1998.0,1998-02-28,4010.0,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,0.000768,0.0,-0.001078,0.002531,-0.006808,0.000000,-0.002361,0.011473,8.5107,NaN
2,9783.0,2.0,1998.0,1998-03-31,4010.0,40101015.0,KEYCORP,-0.056884,0.000000,0.000000,...,0.000844,0.0,-0.000097,0.001550,0.004627,0.000000,0.247055,0.011473,7.9387,NaN
3,9783.0,3.0,1998.0,1998-04-30,4010.0,40101015.0,KEYCORP,-0.057011,0.000000,0.000000,...,0.000941,0.0,-0.016724,0.004338,0.004707,0.000000,-0.002936,0.011641,5.1254,NaN
4,9783.0,4.0,1998.0,1998-05-31,4010.0,40101015.0,KEYCORP,-0.057341,0.000000,0.000000,...,0.000966,0.0,-0.004416,0.002302,0.005910,0.000000,-0.002927,0.011641,-3.9686,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200483,5211.0,1.0,2000.0,2000-02-29,2530.0,25301010.0,MIRAGE RESORTS INC,0.000000,0.075124,0.842431,...,0.000768,0.0,-0.000450,0.002982,-0.047984,0.017857,-0.502211,0.010304,26.0000,14.442975
200484,5211.0,2.0,2000.0,2000-03-31,2530.0,25301010.0,MIRAGE RESORTS INC,0.000000,0.028328,0.636467,...,0.000825,0.0,-0.003625,-0.001308,-0.004308,1.800000,0.496691,0.010304,23.0159,15.146187
200485,5211.0,3.0,2000.0,2000-04-30,2530.0,25301010.0,MIRAGE RESORTS INC,0.000000,0.012931,0.615960,...,0.000824,0.0,-0.008714,0.003107,0.001825,-0.260870,-0.335532,0.024549,5.1613,15.045272
200486,5211.0,4.0,2000.0,2000-05-31,2530.0,25301010.0,MIRAGE RESORTS INC,0.000000,0.008384,0.556918,...,0.000827,0.0,-0.000783,-0.000288,0.001951,0.636364,0.247257,0.024549,2.7607,14.988710


In [4]:
import numpy as np

def stock_price_6(data):
    
    # Group the data by the stock symbol (cusp)
    group = data.groupby('gvkey')
    
    # Create an empty list to store the risk values
    trt6m = []
    
    # Loop through each stock symbol (cusp) and calculate the moving risk
    for name, symbol_data in group:
        # Create an empty list to store the moving risk values
        moving_return_6 = []
        
        datetime_date_x = []
        name_list = []
        # Loop through each date in the stock data
        for i in range(6, len(symbol_data)+1):
            symbol_return = (np.prod((symbol_data['trt1m'].iloc[i-6:i]/100)+1)-1)*100
            datetime_date = symbol_data['date'].iloc[i-1:i].iloc[0]
            moving_return_6.append(symbol_return)
            datetime_date_x.append(datetime_date)
            name_list.append(name)
            trt6m.append([name, symbol_return, datetime_date])
    
    # Create a new dataframe with the calculated risk values
    return_df = pd.DataFrame(trt6m, columns=['gvkey', 'moving_return_6','datetime_date_x'])
    
    # Merge the risk values back into the original dataframe
    #data_with_risk = pd.merge(data, risk_df, on='cusp')
    
    return return_df

In [5]:
return_data_6 = stock_price_6(sp)
return_data_6.head()

,gvkey,moving_return_6,datetime_date_x
0,1013.0,-12.500010,1998-06-30
1,1013.0,75.732980,1998-07-31
2,1013.0,-14.045357,1998-08-31
3,1013.0,-23.357448,1998-09-30
4,1013.0,-23.173348,1998-10-31


In [6]:
modeling_df_6 = pd.merge(sp, return_data_6, left_on=["gvkey", "date"], right_on=["gvkey", "datetime_date_x"], how = 'left')
modeling_df_6 = modeling_df_6.drop(columns=['datetime_date_x'])

In [7]:
modeling_df_6.loc[modeling_df_6['ggroup'] == 1010, 'ggroup'] = 'Energy'
modeling_df_6.loc[modeling_df_6['ggroup'] == 1510, 'ggroup'] = 'Materials'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2010, 'ggroup'] = 'Capital Goods'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2020, 'ggroup'] = 'Commercial & Professional Services'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2030, 'ggroup'] = 'Transportation'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2510, 'ggroup'] = 'Automobiles & Components'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2520, 'ggroup'] = 'Consumer Durables & Apparel'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2530, 'ggroup'] = 'Consumer Services'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2540, 'ggroup'] = 'Media'
modeling_df_6.loc[modeling_df_6['ggroup'] == 2550, 'ggroup'] = 'Retailing'
modeling_df_6.loc[modeling_df_6['ggroup'] == 3010, 'ggroup'] = 'Food & Staples Retailing'
modeling_df_6.loc[modeling_df_6['ggroup'] == 3020, 'ggroup'] = 'Food, Beverage & Tobacco'
modeling_df_6.loc[modeling_df_6['ggroup'] == 3030, 'ggroup'] = 'Household & Personal Products'
modeling_df_6.loc[modeling_df_6['ggroup'] == 3510, 'ggroup'] = 'Health Care Equipment & Services'
modeling_df_6.loc[modeling_df_6['ggroup'] == 3520, 'ggroup'] = 'Pharmaceuticals, Biotechnology & Life Sciences'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4010, 'ggroup'] = 'Banks'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4020, 'ggroup'] = 'Diversified Financials'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4030, 'ggroup'] = 'Insurance'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4510, 'ggroup'] = 'Software & Services'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4520, 'ggroup'] = 'Technology Hardware & Equipment'
modeling_df_6.loc[modeling_df_6['ggroup'] == 4530, 'ggroup'] = 'Semiconductors & Semiconductor Equipment'
modeling_df_6.loc[modeling_df_6['ggroup'] == 5010, 'ggroup'] = 'Telecommunication Services'
modeling_df_6.loc[modeling_df_6['ggroup'] == 5020, 'ggroup'] = 'Media & Entertainment'
modeling_df_6.loc[modeling_df_6['ggroup'] == 5510, 'ggroup'] = 'Utilities'
modeling_df_6.loc[modeling_df_6['ggroup'] == 6010, 'ggroup'] = 'Real Estate'

In [8]:
modeling_df_6

,gvkey,month,year,date,ggroup,gsubind,conm,dpr,PEG_trailing,bm,...,Bank Prime Loan Rate,M2,"Bank Credit, All Commercial Banks",Currency in Circulation,10-Year Treasury Constant Maturity Minus Federal Funds Rate,Rent Inflation Rate,US GDP Growth Rate,trt1m,moving_risk,moving_return_6
0,9783.0,12.0,1997.0,1998-01-31,Banks,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,0.0,-0.008832,-0.001071,-0.001059,0.000000,0.249390,0.011473,-8.2076,NaN,NaN
1,9783.0,1.0,1998.0,1998-02-28,Banks,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,0.0,-0.001078,0.002531,-0.006808,0.000000,-0.002361,0.011473,8.5107,NaN,NaN
2,9783.0,2.0,1998.0,1998-03-31,Banks,40101015.0,KEYCORP,-0.056884,0.000000,0.000000,...,0.0,-0.000097,0.001550,0.004627,0.000000,0.247055,0.011473,7.9387,NaN,NaN
3,9783.0,3.0,1998.0,1998-04-30,Banks,40101015.0,KEYCORP,-0.057011,0.000000,0.000000,...,0.0,-0.016724,0.004338,0.004707,0.000000,-0.002936,0.011641,5.1254,NaN,NaN
4,9783.0,4.0,1998.0,1998-05-31,Banks,40101015.0,KEYCORP,-0.057341,0.000000,0.000000,...,0.0,-0.004416,0.002302,0.005910,0.000000,-0.002927,0.011641,-3.9686,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200483,5211.0,1.0,2000.0,2000-02-29,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.075124,0.842431,...,0.0,-0.000450,0.002982,-0.047984,0.017857,-0.502211,0.010304,26.0000,14.442975,20.574118
200484,5211.0,2.0,2000.0,2000-03-31,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.028328,0.636467,...,0.0,-0.003625,-0.001308,-0.004308,1.800000,0.496691,0.010304,23.0159,15.146187,38.392880
200485,5211.0,3.0,2000.0,2000-04-30,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.012931,0.615960,...,0.0,-0.008714,0.003107,0.001825,-0.260870,-0.335532,0.024549,5.1613,15.045272,39.914141
200486,5211.0,4.0,2000.0,2000-05-31,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.008384,0.556918,...,0.0,-0.000783,-0.000288,0.001951,0.636364,0.247257,0.024549,2.7607,14.988710,61.835670


In [9]:
modeling_df_6['gsubind'].nunique()
# maybe not choosing this one 

149

In [9]:
mv = pd.read_csv('monthly_return.csv',low_memory=False)
fic = mv[['fic','GVKEY']]
gg = modeling_df_6[['ggroup','gvkey']]
fic = fic.rename(columns={'GVKEY': 'gvkey'})

In [10]:
fic = fic.drop_duplicates()
gg = gg.drop_duplicates()

In [11]:
ggmap = pd.merge(gg, fic, on=["gvkey"], how = 'left')
ggmap

,ggroup,gvkey,fic
0,Banks,9783.0,USA
1,Banks,11687.0,USA
2,Retailing,11672.0,USA
3,Banks,4737.0,USA
4,Consumer Services,25434.0,USA
...,...,...,...
1025,"Food, Beverage & Tobacco",12978.0,USA
1026,Consumer Durables & Apparel,35035.0,USA
1027,"Pharmaceuticals, Biotechnology & Life Sciences",11288.0,USA
1028,Utilities,2847.0,USA


In [12]:
ggmap.to_csv('ggmap.csv', index=False)

In [14]:
modeling_df_7 = pd.merge(modeling_df_6, fic, on=["gvkey"], how = 'left')
modeling_df_7

,gvkey,month,year,date,ggroup,gsubind,conm,dpr,PEG_trailing,bm,...,M2,"Bank Credit, All Commercial Banks",Currency in Circulation,10-Year Treasury Constant Maturity Minus Federal Funds Rate,Rent Inflation Rate,US GDP Growth Rate,trt1m,moving_risk,moving_return_6,fic
0,9783.0,12.0,1997.0,1998-01-31,Banks,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,-0.008832,-0.001071,-0.001059,0.000000,0.249390,0.011473,-8.2076,NaN,NaN,USA
1,9783.0,1.0,1998.0,1998-02-28,Banks,40101015.0,KEYCORP,0.000000,0.000000,0.000000,...,-0.001078,0.002531,-0.006808,0.000000,-0.002361,0.011473,8.5107,NaN,NaN,USA
2,9783.0,2.0,1998.0,1998-03-31,Banks,40101015.0,KEYCORP,-0.056884,0.000000,0.000000,...,-0.000097,0.001550,0.004627,0.000000,0.247055,0.011473,7.9387,NaN,NaN,USA
3,9783.0,3.0,1998.0,1998-04-30,Banks,40101015.0,KEYCORP,-0.057011,0.000000,0.000000,...,-0.016724,0.004338,0.004707,0.000000,-0.002936,0.011641,5.1254,NaN,NaN,USA
4,9783.0,4.0,1998.0,1998-05-31,Banks,40101015.0,KEYCORP,-0.057341,0.000000,0.000000,...,-0.004416,0.002302,0.005910,0.000000,-0.002927,0.011641,-3.9686,NaN,NaN,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200483,5211.0,1.0,2000.0,2000-02-29,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.075124,0.842431,...,-0.000450,0.002982,-0.047984,0.017857,-0.502211,0.010304,26.0000,14.442975,20.574118,USA
200484,5211.0,2.0,2000.0,2000-03-31,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.028328,0.636467,...,-0.003625,-0.001308,-0.004308,1.800000,0.496691,0.010304,23.0159,15.146187,38.392880,USA
200485,5211.0,3.0,2000.0,2000-04-30,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.012931,0.615960,...,-0.008714,0.003107,0.001825,-0.260870,-0.335532,0.024549,5.1613,15.045272,39.914141,USA
200486,5211.0,4.0,2000.0,2000-05-31,Consumer Services,25301010.0,MIRAGE RESORTS INC,0.000000,0.008384,0.556918,...,-0.000783,-0.000288,0.001951,0.636364,0.247257,0.024549,2.7607,14.988710,61.835670,USA


In [15]:
modeling_df_7 = modeling_df_7.rename(columns={'moving_return_6': 'trt6m'})
modeling_df_7.to_csv('final_data_V7.csv', index=False)

In [ ]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [20]:
amazon_64768 = return_data[return_data['gvkey']== 64768]
amazon_64768

,gvkey,moving_return_12,datetime_date_x
158636,64768.0,966.390449,1998-12-31
158637,64768.0,1089.195129,1999-01-31
158638,64768.0,898.376823,1999-02-28
158639,64768.0,1107.895456,1999-03-31
158640,64768.0,1025.203828,1999-04-30
...,...,...,...
158909,64768.0,4.329183,2021-09-30
158910,64768.0,11.075958,2021-10-31
158911,64768.0,10.701673,2021-11-30
158912,64768.0,2.376903,2021-12-31


In [24]:
trt1 = (amazon['trt1m'].iloc[0:12]/100)+1
trt1

109820    0.979253
109821    1.305085
109822    1.110792
109823    1.072711
109824    0.960490
109825    2.263830
109826    1.111529
109827    0.755355
109828    1.332836
109829    1.132699
109830    1.518537
109831    1.673177
Name: trt1m, dtype: float64

In [3]:
amazon = sp[sp['conm']== 'AMAZON.COM INC']

In [4]:
amazon.to_csv('amazon.csv', index=False)

In [ ]:
# Load and preprocess the data
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tft.keras import TemporalFusionTransformer

# Set hyperparameters
n_epochs = 100
batch_size = 32
n_time_steps = 60
n_features = 4
hidden_layer_size = 128
n_predictions = 30

# Load training and testing data
train_data = amazon[amazon['date'] < '2018-01-01']# Load your training data
test_data = amazon[amazon['date']>='2018-01-01']# Load your testing data

# Create TFT model
inputs = Input(shape=(n_time_steps, n_features))
tft_layer = TemporalFusionTransformer(hidden_layer_size, output_size=n_predictions)(inputs)
lstm_layer = LSTM(hidden_layer_size)(inputs)
concat_layer = Concatenate()([tft_layer, lstm_layer])
outputs = Dense(n_predictions)(concat_layer)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='mse', optimizer=Adam(lr=0.001))
plot_model(model, show_shapes=True)

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(train_data, epochs=n_epochs, batch_size=batch_size,
                    validation_data=test_data, callbacks=[early_stop])

# Evaluate the model
loss = model.evaluate(test_data)

# Make predictions
predictions = model.predict(test_data)

In [1]:
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data.examples import load_multivariate_time_series_from_file
import pytorch_lightning as pl
from pytorch_forecasting.metrics import SMAPE

# Load data
data = load_multivariate_time_series_from_file()
data

ImportError: cannot import name 'load_multivariate_time_series_from_file' from 'pytorch_forecasting.data.examples' (/opt/anaconda3/lib/python3.9/site-packages/pytorch_forecasting/data/examples.py)

In [ ]:
train_size = 
test_size = 36
train_X, train_y = X[:train_size].dropna(), y[:train_size].dropna()
test_X, test_y = X[train_size:].dropna(), y[train_size:].dropna()